In [2]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.chat_models import ChatOpenAI
from PyPDF2 import PdfReader

In [3]:
load_dotenv()

#using openai api key and loading data
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
data_path = "./data/"

db_directory = "./vector/"

pdf_files= os.listdir(data_path)
print(pdf_files)

def get_pdf_text(data_path, pdf_files):
    
    text = ""

    for pdf_file in pdf_files:
        reader = PdfReader(data_path+pdf_file)
        for page in reader.pages:
            text += page.extract_text()

    return text

text = get_pdf_text(data_path, pdf_files)

['Easy_recipes.pdf', 'recipe-book-1.zp210082.pdf', 'Recipe-Book.pdf', 'Recipes.pdf']
